# Imports

In [1]:
from Arm_Lib import Arm_Device
import time
import cv2
import tensorflow as tf
from keras.models import load_model
import numpy as np
from PIL import Image
import io
# import json
# from tensorflow.keras.models import model_from_json

# Initial Variable Setup

In [2]:
Arm = Arm_Device()

# All Positions

In [4]:
positions = {
    0: [123, 67, 26, 1, 116, 135],
    1: [110, 72, 19, 1, 107, 135],
    2: [95, 72, 18, 2, 87, 135],
    3: [79, 74, 17, 2, 75, 135],
    4: [66, 72, 18, 4, 65, 135],
    5: [70, 62, 32, 4, 67, 135],
    6: [78, 86, 5, 16, 73, 135],
    7: [92, 79, 13, 11, 87, 135],
    8: [105, 67, 28, 5, 102, 135],
    9: [118, 61, 29, 8, 111, 135],
    10: [114, 52, 41, 9, 108, 135],
    11: [105, 54, 41, 7, 98, 135],
    12: [91, 56, 43, 3, 88, 135],
    13: [81, 66, 22, 18, 77, 135],
    14: [73, 50, 46, 5, 66, 135],
    15: [76, 37, 66, 3, 68, 135],
    16: [84, 39, 61, 4, 79, 135],
    17: [95, 48, 47, 12, 87, 135],
    18: [103, 45, 50, 11, 98, 135],
    19: [111, 42, 53, 12, 106, 135],
    20: [109, 30, 72, 11, 105, 135],
    21: [99, 39, 57, 18, 98, 135],
    22: [91, 44, 53, 16, 88, 135],
    23: [85, 30, 73, 8, 81, 135],
    24: [76, 51, 36, 33, 75, 135],
    25: [76, 34, 65, 23, 71, 135],
    26: [83, 35, 62, 23, 80, 135],
    27: [93, 38, 54, 29, 87, 135],
    28: [100, 24, 80, 14, 95, 135],
    29: [106, 28, 70, 22, 104, 135],
    30: [105, 21, 74, 33, 104, 135],
    31: [98, 2, 115, 12, 100, 133],
    32: [92, 2, 115, 12, 92, 132],
    33: [85, 2, 115, 12, 85, 131],
    34: [80, 2, 115, 12, 80, 130],
    'dice': [103, 3, 91, 48, 87, 180],
    'dice_read': [90, 64, 63, 0, 87, 180]
}

# Functions

Function to move the arm to a given position

In [5]:
def moveArm(armObj, array):
    armObj.Arm_serial_servo_write6(array[0], array[1], array[2], array[3], array[4], array[5], 4000)

Function to return to the top (neutral position)

In [8]:
def return_to_top(armObj):
    print("Going up")
    armObj.Arm_serial_servo_write6(90, 90, 90, 90, 90, 180, 2000)

Function to Pick up the Piece

In [9]:
def pick_up(armObj, pick_position):

    # Ensure the gripper is open for pickup
    pick_position[5] = 135
    print(f"Moving to pick position: {pick_position}")
    
    # Move to the pick position
    moveArm(armObj, pick_position)
    time.sleep(5)

    # Close the gripper to pick up the piece
    armObj.Arm_serial_servo_write(6, 180, 3000)
    time.sleep(4)
    print("Picking up object...")

    # Go back to the top
    return_to_top(armObj)
    time.sleep(3)

Function to put the piece down again

In [10]:
def put_down(armObj, down_position):
    print(f"Moving to down position: {down_position}")

    # Move to the put down position
    down_position[5] = 180
    moveArm(armObj, down_position)
    time.sleep(5)
    
    # Open the gripper to drop the piece
    armObj.Arm_serial_servo_write(6, 135, 3000)
    time.sleep(4)
    print("Dropping Object...")

    # Go back to the top
    return_to_top(armObj)
    time.sleep(3)

    # Open the Gripper again to get ready for next pickup
    armObj.Arm_serial_servo_write(6, 135, 3000)

Function to Take a Picture

In [11]:
def take_picture():
    # Open the camera (default is usually 0)
    cap = cv2.VideoCapture(0)
    
    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Could not access the camera.")
        
    else:
        # Capture a single frame from the camera
        ret, frame = cap.read()
    
        if ret:
            # Encode the image as jpg
            ret, jpg_image = cv2.imencode('.jpg', frame)
            
        else:
            print("Error: Could not capture image.")
    
    # Release the camera and close any OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

    # Return the jpg image in byte format to easily load into AI model
    return jpg_image.tobytes()

Function to roll the dice

In [12]:
def roll_dice():

    # Move to the dice position to roll the dice
    moveArm(Arm, positions['dice'])
    time.sleep(4)

    # move to the 'dice read' position to see the dice properly
    moveArm(Arm, positions['dice_read'])
    time.sleep(5)

    # Take the picture for analysis
    image = take_picture()

    # Go back to the top and open the gripper to get ready to move the piece
    return_to_top(Arm)
    time.sleep(5)
    Arm.Arm_serial_servo_write(6, 135, 2000)
    time.sleep(3)

    # Return the image (Still in byte format)
    return image


Function to make a prediction on a given image

In [40]:
def predict_image(model, jpg_image):

    # Declare class names as the model is loaded from .h5 file
    class_names = ['1', '2', '3', '4', '5', '6']

    # Load the given image (image is in byte format) and transform it into a numpy array
    image = Image.open(io.BytesIO(jpg_image))
    image_array = np.array(image)

    # Resize the image to 180 x 180 as the AI model uses that size
    image_array = cv2.resize(image_array, (180, 180))
    image_array = image_array / 255.0

    # Expand the dimenstions of the array to suit the format required by AI model
    image_array = np.expand_dims(image_array, axis=0)

    # Make the prediction
    prediction = model.predict(image_array)

    # Isolate the important variables from the prediciton
    confidence_scores = prediction[0]
    max_index = np.argmax(confidence_scores)
    class_name = class_names[max_index]
    confidence_percentage = confidence_scores[max_index] * 100

    # _________________________________________
    score = tf.nn.softmax(prediction[0])
    print(f"CLASS: {class_names[(np.argmax(score))]}, SCORE: {100 * np.max(score)}%")
    #_______________________________________________________
    # Print out the result and return the classname in int form to use for making moves
    # print(f"Predicted class: {class_name} ({round(100 * np.max(score)), 2}% confidence)")
    return int(class_name)


Functions to check for Snakes & Ladders

In [24]:
def check_for_snakes(robot_position):

    # Set the snake positions
    snake_positions = [7, 10, 21, 24, 33]

    # Set the places the robot has to move back to if it lands on a snake
    new_locations = {
        7: 4,
        10: 2,
        21: 13,
        24: 6,
        33: 19
    }
    
    # Check if the robot has landed on a snake and return the new location if it did land on a snake
    if robot_position in snake_positions:
        print("Snake!")
        return new_locations[robot_position]
    else:
        print("No Snake :)")
        return False
        

In [15]:
def check_for_ladders(robot_position):

    # Set the snake positions
    ladder_positions = [1, 5, 11, 15, 20]

    # Set the places the robot moves the piece to if it lands on a ladder
    new_locations = {
        1: 12,
        5: 16,
        11: 22,
        15: 23,
        20: 31
    }
    
    # Check if the robot has landed on a snake and return the new location if it did land on a snake
    if robot_position in ladder_positions:
        print("Ladder!")
        return new_locations[robot_position]
    else:
        print("No ladder :(")
        return False

# MAIN FUNCTION

In [42]:
def main():
    # Load the Robotic Arm to control the arm
    Arm = Arm_Device()
    
    # Load the AI model for predictions
    model = load_model('dice_detection_final.h5')

    # Initialise the Robot's position and set win status to false
    current_position = 0
    win = False
    
    # Play until it has won
    while win == False:

        # First roll the dice to know how much to move
        dice_roll_image = roll_dice()

        # # Make AI Prediction on the dice roll
        number_of_moves = predict_image(model, dice_roll_image)

        # Calculate the new position to be in
        new_position = current_position + number_of_moves

        # Make sure the new position is still on the board
        if new_position < 34:

            # Move to the piece to the new position
            pick_up(Arm, positions[current_position])
            put_down(Arm, positions[new_position])
    
            # Set the current position to the new position
            current_position = new_position

            # Check if there is a snake or a ladder
            snake = check_for_snakes(current_position)
            ladder = check_for_ladders(current_position)

            # Move accordingly based on the check of snakes and ladders
            # Note that the checking functions return the NEW POSITION of the Robot
            if snake:
                pick_up(Arm, positions[current_position])
                put_down(Arm, positions[snake])
                current_position = snake
            if ladder:
                pick_up(Arm, positions[current_position])
                put_down(Arm, positions[ladder])
                current_position = ladder

        # If the new position is >=34, then the game is over
        else:
            
            # Move the robot's piece to the ending square and set the win flag to true
            pick_up(Arm, positions[current_position])
            put_down(Arm, positions[34])
            win = True
        

In [ ]:
main()

Going up
1/1 [==============================] - 0s 291ms/step
CLASS: 6, SCORE: 26.80204212665558
Predicted class: 6 ((27, 2)% confidence)
Moving to pick position: [123, 67, 26, 1, 116, 135]
Picking up object...
Going up
Moving to down position: [78, 86, 5, 16, 73, 135]
Dropping Object...
Going up
No Snake :)
No ladder :(
Going up
1/1 [==============================] - 0s 86ms/step
CLASS: 6, SCORE: 26.306328177452087
Predicted class: 6 ((26, 2)% confidence)
Moving to pick position: [78, 86, 5, 16, 73, 135]
Picking up object...
Going up
Moving to down position: [91, 56, 43, 3, 88, 135]
Dropping Object...
Going up
No Snake :)
No ladder :(
Going up
1/1 [==============================] - 0s 87ms/step
CLASS: 5, SCORE: 26.23499631881714
Predicted class: 5 ((26, 2)% confidence)
Moving to pick position: [91, 56, 43, 3, 88, 135]
